In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D,\
     Flatten, BatchNormalization, AveragePooling2D, Dense, Activation, Add, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

import random
import shutil
import cv2
import os
from PIL import Image

In [ ]:
# Increasing the image size didn't result in increasing the training accuracy
IMAGE_WIDTH = 600    
IMAGE_HEIGHT = 800
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Creates the training and validation data generators
  
  Args:
    TRAINING_DIR (string): directory path containing the training images
    VALIDATION_DIR (string): directory path containing the testing/validation images
    
  Returns:
    train_generator, validation_generator - tuple containing the generators
  """
  ### START CODE HERE

  # Instantiate the ImageDataGenerator class (don't forget to set the arguments to augment the images)
  train_datagen = ImageDataGenerator(rescale=1/255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
#                                      brightness_range=[0.7,1.5],
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     fill_mode='nearest')

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=10,
                                                      class_mode='categorical',
                                                      target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator(rescale=1/255)

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=10,
                                                                class_mode='categorical',
                                                                target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
  ### END CODE HERE
  return train_generator, validation_generator

In [ ]:
# Test generators
ROOT_DIR = "/kaggle/input/oil-palm-class"
TRAINING_DIR = os.path.join(ROOT_DIR, "train")
VALIDATION_DIR = os.path.join(ROOT_DIR, "valid")

train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

In [ ]:
pre_trained_model = MobileNetV2(input_shape =(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS), include_top = False)
for layer in pre_trained_model.layers:
    layer.trainable = False  
pre_trained_model.summary()

In [ ]:
#MobileNet Model
x=Flatten()(pre_trained_model.output)

#Fully Connection Layers
# FC1
x=Dense(512, activation="relu")(x)
x=Dense(256, activation="relu")(x)
#Dropout to avoid overfitting effect
x=Dropout(0.2)(x)

# FC2
x=Dense(128, activation="relu")(x)


#output layer
x=Dense(6,activation="sigmoid")(x)


model = Model(pre_trained_model.input,x)

model.summary()

In [ ]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['acc', f1_m])

In [ ]:
def earlystop(mode):
  if mode=='acc':
    estop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=30, mode='max')
  elif mode=='loss':
    estop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, mode='min')
  return estop 

earlystop = earlystop('acc')

checkpoint = tf.keras.callbacks.ModelCheckpoint("/kaggle/working/best_model_train3",
                                            save_best_only=True, monitor='val_loss', mode="min")

In [2]:
batch_size = 20
steps_per_epoch = 374/batch_size
validation_steps = 360/batch_size
classifier_history = model.fit(train_generator, 
                                epochs=200, 
                                steps_per_epoch=steps_per_epoch,
                                validation_steps=validation_steps, 
                                validation_data=validation_generator, 
                                callbacks=[checkpoint])

NameError: name 'model' is not defined

In [3]:
import os
os.chdir(r'/kaggle/working')

!tar -czf best_model_train3.tar.gz best_model_train3

from IPython.display import FileLink

FileLink(r'best_model_train3.tar.gz')

/kaggle/working/best_model_train3.tar.gz